In [4]:
# imports
import csv
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [ ]:
'''Im going to need a new set of track data, as much as i can as i believe 
the more data the more accurate the model im going to download a  10,000 song playlist from spotify api
that spans the widest range of genres and decades'''

In [8]:
# Authorise Api token
client_id = '523da5013ebf451dbb4b9a05bb06f06d'
client_secret = 'b7167442889b40dd906acb9fce3b7aa7'
client_credentials_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials_manager)

In [9]:
'''create function to get data'''

# had an issue where it only returned one page of results so needed to create a way of keeping reading next
# page 
def get_playlist_tracks(username, playlist_id):
    #this section solves the pages problem while there is still a 'next' option the code will keep adding
    # 'items'(tracks) to the results
    results = sp.user_playlist_tracks(username,playlist_id)
    tracks = results['items']
    while results['next']:
        results = sp.next(results)
        tracks.extend(results['items'])
    results = tracks    
#create empty dictionarys
    track_ids = []
    track_titles = []
    track_artist = []
    
    for i in range(len(results)):
        print(i) # Counter
        if i == 0:
            track_ids = results[i]['track']['id']
            track_titles = results[i]['track']['name']
            #list of artists
            artist_list = []
            for artist in results[i]['track']['artists']:
                artist_list= artist['name']
                track_artist = artist_list
# use track id's to find audio features
            features = sp.audio_features(track_ids)
    #create a new df with tracks features
            features_df = pd.DataFrame(data=features, columns=features[0].keys())
            features_df['title'] = track_titles
            features_df['all_artists'] = track_artist
            features_df = features_df[['id', 'title', 'all_artists',
                                       'danceability', 'energy', 'key', 'loudness',
                                       'mode', 'acousticness', 'instrumentalness',
                                       'liveness', 'valence', 'tempo',
                                       'duration_ms', 'time_signature']]
            continue
        else:         #put all together into DF
            try:
                track_ids = results[i]['track']['id']
                track_titles = results[i]['track']['name']
                artist_list = []
                for artist in results[i]['track']['artists']:
                    artist_list= artist['name']
                track_artist = artist_list
                features = sp.audio_features(track_ids)
                new_row = {'id':[track_ids],
               'title':[track_titles],
               'all_artists':[track_artist],
               'danceability':[features[0]['danceability']],
               'energy':[features[0]['energy']],
               'key':[features[0]['key']],
               'loudness':[features[0]['loudness']],
               'mode':[features[0]['mode']],
               'acousticness':[features[0]['acousticness']],
               'instrumentalness':[features[0]['instrumentalness']],
               'liveness':[features[0]['liveness']],
               'valence':[features[0]['valence']],
               'tempo':[features[0]['tempo']],
               'duration_ms':[features[0]['duration_ms']],
               'time_signature':[features[0]['time_signature']]
               }

                dfs = [features_df, pd.DataFrame(new_row)]
                features_df = pd.concat(dfs, ignore_index = True)
            except:
                continue
                
    return features_df

In [ ]:
# run function and save to csv file
# takes a while! 10,000 is alot :)

# playlist_id="7A2TXg191J1S4LPesbwSFK"
# username = 'Mark Hudson'
# SpotifyMachineLearning_df = get_playlist_tracks(username, playlist_id)
# SpotifyMachineLearning_df.to_csv('../../datasets/SpotifyMachineLearningData.csv',index = False)

In [11]:
# pd.read_csv('../../datasets/SpotifyMachineLearningData.csv')
# unhash to read

,id,title,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6JNJERZGJwDVgkmbohBw7u,Self Control,Laura Branigan,0.814,0.686,6,-12.172,0,0.21900,0.003820,0.1550,0.778,106.679,246440,4
1,0cTwMCvozZBj8jCq6o27nk,Come Dancing,The Kinks,0.491,0.751,7,-11.549,1,0.23000,0.000012,0.0666,0.916,152.430,236735,4
2,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,The Killers,0.352,0.911,1,-5.230,1,0.00121,0.000000,0.0995,0.236,148.033,222973,4
3,4ih3dyFZoeTdaeJW9mPbOI,Mr. Vain - Original Radio Edit,Culture Beat,0.703,0.997,9,-7.666,0,0.07260,0.023500,0.1230,0.391,132.872,257467,4
4,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,0.516,0.595,5,-4.630,1,0.02750,0.000000,0.1090,0.222,119.972,340707,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,4JBiO7wRnE6ueszEUpo347,Pointless,Lewis Capaldi,0.325,0.590,4,-3.943,1,0.06860,0.000000,0.1040,0.278,180.449,235752,3
9996,4DHcnVTT87F0zZhRPYmZ3B,Flowers,Miley Cyrus,0.707,0.681,0,-4.325,1,0.06320,0.000005,0.0322,0.646,117.999,200455,4
9997,48UKTR66uUOT9LaUvooTNx,The Kind of Love We Make,Luke Combs,0.608,0.745,1,-4.130,1,0.02260,0.000006,0.0942,0.464,102.025,224427,4
9998,3nhGk6VnrDHy67pXvMhdPa,SPACE MAN,Sam Ryder,0.467,0.828,11,-2.220,1,0.18100,0.000000,0.1020,0.334,79.978,217320,4


In [6]:
ML_df = pd.read_csv('../../datasets/SpotifyMachineLearningData.csv')
ML_df.describe()

,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,1.000000e+04,10000.000000
mean,0.581101,0.667234,5.228300,-8.286231,0.715600,0.240124,0.048044,0.186766,0.598684,122.125779,2.321251e+05,3.953500
std,0.154046,0.214047,3.559782,3.583167,0.451151,0.279422,0.156270,0.151245,0.243286,26.874332,6.471804e+04,0.262573
min,0.063700,0.006280,0.000000,-28.656000,0.000000,0.000001,0.000000,0.009670,0.032000,50.786000,7.729300e+04,1.000000
25%,0.480000,0.527000,2.000000,-10.632250,0.000000,0.018800,0.000000,0.088600,0.405000,103.255250,1.912602e+05,4.000000
50%,0.592000,0.703000,5.000000,-7.666000,1.000000,0.111000,0.000038,0.128000,0.622000,120.876000,2.238400e+05,4.000000
75%,0.692000,0.840000,9.000000,-5.491750,1.000000,0.393000,0.003412,0.249000,0.807000,135.956750,2.607502e+05,4.000000
max,0.980000,0.999000,11.000000,1.342000,1.000000,0.996000,0.988000,0.988000,0.994000,216.020000,1.014787e+06,5.000000


In [7]:
testML_df = ML_df[['id','title','all_artists','danceability','energy','key','loudness','mode','acousticness','instrumentalness','liveness','valence','tempo','duration_ms','time_signature']]
testML_df.head(10)

,id,title,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6JNJERZGJwDVgkmbohBw7u,Self Control,Laura Branigan,0.814,0.686,6,-12.172,0,0.219000,0.003820,0.1550,0.778,106.679,246440,4
1,0cTwMCvozZBj8jCq6o27nk,Come Dancing,The Kinks,0.491,0.751,7,-11.549,1,0.230000,0.000012,0.0666,0.916,152.430,236735,4
2,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,The Killers,0.352,0.911,1,-5.230,1,0.001210,0.000000,0.0995,0.236,148.033,222973,4
3,4ih3dyFZoeTdaeJW9mPbOI,Mr. Vain - Original Radio Edit,Culture Beat,0.703,0.997,9,-7.666,0,0.072600,0.023500,0.1230,0.391,132.872,257467,4
4,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,0.516,0.595,5,-4.630,1,0.027500,0.000000,0.1090,0.222,119.972,340707,4
5,3nvuPQTw2zuFAVuLsC9IYQ,Nothing Compares 2 U,Sinéad O'Connor,0.511,0.574,5,-7.016,1,0.042500,0.000023,0.1050,0.161,119.917,280040,4
6,63OQupATfueTdZMWTxW03A,Karma Police,Radiohead,0.360,0.501,7,-9.129,1,0.063800,0.000093,0.1720,0.324,74.807,264067,4
7,4hy4fb5D1KL50b3sng9cjw,Smells Like Teen Spirit,Nirvana,0.480,0.875,1,-5.898,1,0.000016,0.000751,0.1030,0.826,116.754,300827,4
8,2JO3HwMRPeya8bXbtbyPcf,Love Will Tear Us Apart,Joy Division,0.443,0.956,2,-6.198,1,0.000052,0.825000,0.1350,0.909,146.754,206867,4
9,78chqvzuLt8SewW28b3WJP,Last Nite,The Strokes,0.618,0.904,0,-5.691,1,0.021500,0.000305,0.0860,0.768,103.968,197533,4


In [8]:
'''I will need to create my output data based on my mood theory'''
test_df = testML_df.sample(100)

condition_1 = (test_df['valence'] > 0.5)
condition_2 = (test_df['valence'] < 0.5)
condition_3 = (test_df['energy'] > 0.5)
condition_4 = (test_df['energy']< 0.5)

# happy = condition_1 & condition_3
# happy
# sad = condition_2 & condition_4
# #sad
# calm = condition_1 & condition_4
# # print calm
# agitated = condition_2 & condition_3
# print agitated

# unhash to view different track classifications
# test_df[(test_df['valence'] > 0.5) & (test_df['energy'] > 0.5)] # happy
# test_df[(test_df['valence'] < 0.5) & (test_df['energy'] < 0.5)] # sad
# test_df[(test_df['valence'] > 0.5) & (test_df['energy'] < 0.5)] # calm
test_df[(test_df['valence'] < 0.5) & (test_df['energy'] > 0.5)] # agitated

,id,title,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
3,4ih3dyFZoeTdaeJW9mPbOI,Mr. Vain - Original Radio Edit,Culture Beat,0.703,0.997,9,-7.666,0,0.07260,0.023500,0.1230,0.3910,132.872,257467,4
4690,64I0O6ZipYryKte1cHA2EY,End Credits,Plan B,0.324,0.783,1,-4.808,0,0.00929,0.011700,0.0888,0.0788,116.704,200600,3
9464,3yi27cGUFPTQUdWZV9nV67,December,Collective Soul,0.684,0.645,0,-5.707,1,0.58900,0.004320,0.1800,0.3660,121.995,285000,4
6733,63VDd4PRMwWDoUhxxbE6ND,Walking On The Milky Way,Orchestral Manoeuvres In The Dark,0.448,0.760,2,-6.557,1,0.03770,0.000244,0.1130,0.2520,81.648,278893,4
2110,48Hb8h1VlqQDqQHcjkCRK4,Something I Need,Ben Haenow,0.618,0.774,5,-5.676,1,0.04730,0.000000,0.1560,0.4420,100.993,225933,4
6178,0FDzzruyVECATHXKHFs9eJ,A Sky Full of Stars,Coldplay,0.545,0.675,6,-6.474,1,0.00617,0.001970,0.2090,0.1620,124.970,267867,4
3599,4fKhRRYn7F5shZItjJkPJU,Watching The Wheels - Remastered 2010,John Lennon,0.607,0.612,0,-8.666,1,0.40800,0.000005,0.3200,0.2720,82.933,239933,4
2463,4E5P1XyAFtrjpiIxkydly4,Replay,Iyaz,0.706,0.751,9,-6.323,1,0.17300,0.000000,0.1680,0.1950,91.031,182307,4
5495,3jZomvOBa5qfTo5HkqI1p5,One Way or Another (Teenage Kicks),One Direction,0.489,0.867,4,-3.121,0,0.02500,0.000000,0.5860,0.4090,162.131,157293,4
8664,7ra0Uf8XtNk8Uky4gip5yW,Singing In My Sleep,Semisonic,0.448,0.959,4,-5.502,1,0.00460,0.000102,0.3190,0.3790,136.001,270933,4


In [21]:
# seems to output good results now to create a function to add mood output
# create mood column
def mood_category(valence,energy):
    if valence > 0.5 and energy > 0.5:
        mood = 'happy'
    elif valence < 0.5 and energy < 0.5:
        mood = 'sad'
    elif valence > 0.5 and energy < 0.5:
        mood = 'calm'
    else: 
        mood = 'agitated'
    return mood


# test_df['mood'] = test_df['valence'],test_df['energy'].apply(mood_category) 
# didnt work due to two inputs found method online using lambda to input two columns into function


test_df['mood'] = test_df.apply(lambda x: mood_category(x['valence'], x['energy']), axis=1)
test_df[['title', 'mood']]

,title,mood
5467,Life Is A Rollercoaster,happy
2717,Anotherloverholenyohead,happy
6015,Rebel Rebel - 2016 Remaster,agitated
9493,Falling Into You,happy
1623,Mellow Yellow - 2005 Remastered Version,calm
...,...,...
6633,Twilight Zone,happy
1696,Saved By The Bell,sad
3257,All Out of Love,sad
5357,All This Time,agitated


In [22]:
# test_df

,id,title,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,mood
5467,5lQfI9cYzezWGvqwrWkzqh,Life Is A Rollercoaster,Ronan Keating,0.655,0.790,0,-8.928,1,0.10100,0.000155,0.3600,0.858,118.979,234827,4,happy
2717,1M9LnSZjeyLKn0GwT1Xg27,Anotherloverholenyohead,Prince,0.839,0.526,9,-12.717,1,0.29400,0.000599,0.3250,0.863,103.974,240693,4,happy
6015,2EC9IJj7g0mN1Q5VrZkiYY,Rebel Rebel - 2016 Remaster,David Bowie,0.635,0.686,9,-15.648,1,0.20900,0.017600,0.2820,0.464,125.988,274747,4,agitated
9493,7KWMxUH7lORLTob6aMpXBU,Falling Into You,Céline Dion,0.371,0.505,2,-9.052,0,0.38200,0.000000,0.1150,0.564,193.246,258240,4,happy
1623,677emLGTmJ3058RJKVFXtP,Mellow Yellow - 2005 Remastered Version,Donovan,0.785,0.300,2,-10.075,1,0.31300,0.000025,0.0648,0.779,106.656,223787,4,calm
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6633,1miXjv3X34Q9w7wDZoBnDz,Twilight Zone,Golden Earring,0.656,0.756,11,-6.516,0,0.00785,0.000035,0.0927,0.542,118.560,478107,4,happy
1696,0k1HlttWBYieN78qNrJ400,Saved By The Bell,Robin Gibb,0.155,0.298,0,-13.805,1,0.43100,0.000051,0.2140,0.296,91.058,186893,3,sad
3257,7dQC53NiYOY9gKg3Qsu2Bs,All Out of Love,Air Supply,0.513,0.262,0,-16.875,1,0.31300,0.000006,0.4560,0.357,108.383,243000,4,sad
5357,3VLEQvzk2IqoWbHpjnFuPK,All This Time,Michelle McManus,0.528,0.639,6,-4.879,1,0.61400,0.000005,0.1110,0.308,128.231,262293,4,agitated


In [ ]:
# function works as expected so i can use it to add mood column to test data for Machine learning.

# Machine learning

In [28]:
# new imports
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import joblib
from sklearn import tree

In [23]:
# read data from CSV file
df = pd.read_csv('../../datasets/SpotifyMachineLearningData.csv')
# check loaded
df.head()

,id,title,all_artists,danceability,energy,key,loudness,mode,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,6JNJERZGJwDVgkmbohBw7u,Self Control,Laura Branigan,0.814,0.686,6,-12.172,0,0.21900,0.003820,0.1550,0.778,106.679,246440,4
1,0cTwMCvozZBj8jCq6o27nk,Come Dancing,The Kinks,0.491,0.751,7,-11.549,1,0.23000,0.000012,0.0666,0.916,152.430,236735,4
2,003vvx7Niy0yvhvHt4a68B,Mr. Brightside,The Killers,0.352,0.911,1,-5.230,1,0.00121,0.000000,0.0995,0.236,148.033,222973,4
3,4ih3dyFZoeTdaeJW9mPbOI,Mr. Vain - Original Radio Edit,Culture Beat,0.703,0.997,9,-7.666,0,0.07260,0.023500,0.1230,0.391,132.872,257467,4
4,5Ohxk2dO5COHF1krpoPigN,Sign of the Times,Harry Styles,0.516,0.595,5,-4.630,1,0.02750,0.000000,0.1090,0.222,119.972,340707,4


In [24]:
# check for null values
len(df) - df.count()

id                  0
title               0
all_artists         0
danceability        0
energy              0
key                 0
loudness            0
mode                0
acousticness        0
instrumentalness    0
liveness            0
valence             0
tempo               0
duration_ms         0
time_signature      0
dtype: int64

In [25]:
# add mood column
def mood_category(valence,energy):
    if valence > 0.5 and energy > 0.5:
        mood = 'happy'
    elif valence < 0.5 and energy < 0.5:
        mood = 'sad'
    elif valence > 0.5 and energy < 0.5:
        mood = 'calm'
    else: 
        mood = 'agitated'
    return mood


# test_df['mood'] = test_df['valence'],test_df['energy'].apply(mood_category) 
# didnt work due to two inputs found method online using lambda to input two columns into function


df['mood'] = df.apply(lambda x: mood_category(x['valence'], x['energy']), axis=1)
df[['title', 'mood']].head


<bound method NDFrame.head of                                  title      mood
0                         Self Control     happy
1                         Come Dancing     happy
2                       Mr. Brightside  agitated
3       Mr. Vain - Original Radio Edit  agitated
4                    Sign of the Times  agitated
...                                ...       ...
9995                         Pointless  agitated
9996                           Flowers     happy
9997          The Kind of Love We Make  agitated
9998                         SPACE MAN  agitated
9999  Young Love (feat. Laura Marling)     happy

[10000 rows x 2 columns]>

In [26]:
# remove non numerical data
df_cleaned = df.drop(columns = ['id','title','all_artists'])

In [29]:
# split data set input/output split 80/20 test size
X = df_cleaned.drop(columns = ['mood'])
y = df_cleaned['mood']
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

In [31]:
# create model using algorithm decision tree
model = DecisionTreeClassifier()
model.fit(X,y)
predictions = model.predict([[0.325, 0.590, 4, -3.943, 1, 0.06860, 0.000000, 0.1040, 0.278, 180.449, 235752, 3],[0.707, 0.681, 0, -4.325, 1, 0.06320, 0.000005, 0.0322, 0.646, 117.999,200455, 4]])
# quick test should output agitated and happy
predictions
# it works in theory now lets train  the model

C:\Users\amyem\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:439: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(


array(['agitated', 'happy'], dtype=object)

In [32]:
# train model

X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.2)

model = DecisionTreeClassifier()
model.fit(X_train,y_train)

predictions = model.predict(X_test)

score = accuracy_score(y_test, predictions)
score


1.0

In [36]:
#save model
model = DecisionTreeClassifier()
model.fit(X,y)

joblib.dump(model,'spotify_mood_classifier.joblib')

# predictions = model.predict([[0.325, 0.590, 4, -3.943, 1, 0.06860, 0.000000, 0.1040, 0.278, 180.449, 235752, 3],[0.707, 0.681, 0, -4.325, 1, 0.06320, 0.000005, 0.0322, 0.646, 117.999,200455, 4]])
# # quick test should output agitated and happy
# predictions

['spotify_mood_classifier.joblib']

In [37]:
# to view decision tree

X = df_cleaned.drop(columns = ['mood'])
y = df_cleaned['mood']

model = DecisionTreeClassifier()
model.fit(X,y)




tree.export_graphviz(model,out_file = 'spotify_mood_classifier.dot',
                     feature_names = ['danceability', 'energy', 'key', 'loudness', 'mode', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms','time_signature'],
                     class_names = sorted(y.unique()),
                     label = 'all',
                     rounded = True,
                     filled = True)
    # creates a .dot file that can be viewed in pycharm

<img src="../images/spotify_mood_classifier.png">

We can see the model has learned to categorise by mood using my parameters, pretty cool :)

# Model trained back to spotify data mood analysis ipynb to feed it the data we scraped.